In [3]:
import pandas as pd
import numpy as np
import io
import dvc.api
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os

/home/owon/anaconda3/envs/issue/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [2]:
df_train = pd.read_csv('../data/df_train.csv')
df_test = pd.read_csv('../data/df_test.csv')

In [5]:
from data_cleaner import Clean_df

SyntaxError: invalid syntax (data_cleaner.py, line 165)